<a href="https://colab.research.google.com/github/CarolGitonga/Natural-Language-Processing/blob/main/Basic_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install contractions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 10.2 MB/s eta 0:00:00


In [2]:
# Load dependencies
import numpy as np
import re
import requests
import nltk
nltk.download('gutenberg', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('omw-1.4', quiet=True)
nltk.download('averaged_perceptron_tagger', quiet=True)
from nltk.corpus import gutenberg, stopwords, wordnet
from nltk import FreqDist, word_tokenize, pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.collocations import BigramCollocationFinder, TrigramCollocationFinder
from nltk.util import trigrams
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from collections import Counter
import spacy
from spacy.tokenizer import Tokenizer
import string
import pathlib
import math
import contractions
from itertools import islice

In [3]:
def request_gutenberg(url):

  # Make a request to the url
  response = requests.get(url)

  # check if the request was successful
  if response.status_code == 200:
    book = response.text

  else:
    print(f"Failed to retrieve the text version. Status code: {response.status_code}")

  # remove unwanted new line and tab characters from the text, replacing with whitespace
  unwanted_chars = ["\n", "\r", "\d", "\t"]
  for char in unwanted_chars:
      book = book.replace(char, " ")

  return book

# URLs of the books to be used in this lab
g_gatsby_url = "https://www.gutenberg.org/cache/epub/64317/pg64317.txt"
huckleberry_finn_url = "https://www.gutenberg.org/cache/epub/76/pg76.txt"
sherlock_holmes_url = "https://www.gutenberg.org/files/1661/1661-0.txt"

# Make requests for each book
g_gatsby = request_gutenberg(g_gatsby_url)[1494:277912] # Remove introduction and footnotes
h_berry = request_gutenberg(huckleberry_finn_url)[9528:-18862]
s_holmes = request_gutenberg(sherlock_holmes_url)[1508:-18859]

books = {
    "The Great Gatsby": g_gatsby,
    "Adventures of Huckleberry Finn": h_berry,
    "The Adventures of Sherlock Holmes": s_holmes
    }

# Preview
for key, val in books.items():
  print(f"{key}:")
  print(val[:500])
  print("="*100)

The Great Gatsby:
I    In my younger and more vulnerable years my father gave me some advice  that I’ve been turning over in my mind ever since.    “Whenever you feel like criticizing anyone,” he told me, “just  remember that all the people in this world haven’t had the advantages  that you’ve had.”    He didn’t say any more, but we’ve always been unusually communicative  in a reserved way, and I understood that he meant a great deal more  than that. In consequence, I’m inclined to reserve all judgements, a  habi
Adventures of Huckleberry Finn:
CHAPTER I.      You don’t know about me without you have read a book by the name of The  Adventures of Tom Sawyer; but that ain’t no matter. That book was made  by Mr. Mark Twain, and he told the truth, mainly. There was things  which he stretched, but mainly he told the truth. That is nothing. I  never seen anybody but lied one time or another, without it was Aunt  Polly, or the widow, or maybe Mary. Aunt Polly—Tom’s Aunt Polly, she  is—and Mar